In [ ]:
# load binary file
import numpy as np
from scipy import ndimage
from scipy.ndimage import filters
from scipy.ndimage import gaussian_filter
import scipy.sparse as sparse
import time
import matplotlib.pyplot as plt
import math
import scipy.io as sio
from matplotlib.colors import hsv_to_rgb
%matplotlib inline
import celldetect, imp

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops0 = {
        'navg_frames_svd': 5000,
        'nsvd_for_roi': 1000,
        'ratio_neuropil': 5,
        'tile_factor': 1,        
        'threshold_scaling': 1,
        'Vcorr': [],
        'allow_overlap': False,
        'inner_neuropil_radius': 2, 
        'outer_neuropil_radius': np.inf, 
        'min_neuropil_pixels': 350, 
        'ratio_neuropil_to_cell': 4,     
        'nframes': 1,
        'diameter': 12
    }

ops1 = np.load('H:/DATA/2017-10-13\suite2p\plane0\ops.npy').item()

ops = {**ops0, **ops1} # replace default values with saved ones
ops['reg_file'] = 'H:/DATA/2018_03_02/1/TX4_2017-10-13_4_plane1.bin'

In [ ]:
ds = np.load('H:\DATA\TX4\ds.npy')
print(np.corrcoef(ds[:,0], ops['yoff']))
print(np.corrcoef(ds[:,1], ops['xoff']))

#plt.plot(ds[:,1], ops['xoff'], 'r.')

plt.plot(ds[:,0] - ops['yoff'])

In [ ]:
imp.reload(celldetect)

ops['reg_file'] = 'H:/DATA/2018_03_02/1/TX4_2017-10-13_4_plane1.bin'
U,sdmov      = celldetect.getSVDdata(ops)
S, StU , StS = celldetect.getStU(ops, U)

In [ ]:
plt.imshow(S[1,:].reshape((U.shape[1], U.shape[2])))

In [ ]:
nsvd, Lyc, Lxc = U.shape

d0 = ops['diameter']

sig = np.ceil(d0 / 4) # smoothing constant

# make array of radii values of size (2*d0+1,2*d0+1)

rs,dy,dx     = celldetect.circleMask(d0)

ncell = int(1e4)
mPix = -1*np.ones((dx.size, ncell), np.int32)
mLam = np.zeros((dx.size, ncell), np.float32)

it = 0

ncells = 0
r = np.random.random((ncell,))

L = sparse.lil_matrix((1,Lyc*Lxc), dtype=np.float32)
nsvd = U.shape[0]
nbasis = S.shape[0]
LtU = np.zeros((0,nsvd), np.float32)
LtS = np.zeros((0,nbasis), np.float32)

Ucell = U
err = (Ucell**2).mean()
print(err)

# regress maps onto basis functions and subtract neuropil contribution
neu = np.linalg.solve(StS, StU)
Ucell = U -  np.reshape(neu.transpose() @ S, U.shape)

def sub2ind(array_shape, rows, cols):
    inds = rows * array_shape[1] + cols
    return inds

err = (Ucell**2).mean()
print(err)

In [ ]:
i0 = tic()
it = 0

while it<10:
    # smooth residual
    us = gaussian_filter(Ucell, [0., sig, sig],  mode='wrap')

    # compute log variance at each location
    V  = (us**2).mean(axis=0)
    um = (Ucell**2).mean(axis=0)
    um = gaussian_filter(um, [sig, sig],  mode='wrap')
    V  = V / um

    V = V.astype('float64')

    if it==0:
        # perform morphological opening on V to normalize brightness
        vrem   = celldetect.morphOpen(V, rs<=d0)
        
    V      = V - vrem
    
    if it==0:        
        # find indices of all maxima in +/- 1 range
        # use the median of these peaks to decide stopping criterion
        maxV   = filters.maximum_filter(V, footprint= (rs<=1.5))

        imax   = V > (maxV - 1e-10)
        peaks  = V[imax]
        thres  = ops['threshold_scaling'] * np.median(peaks[peaks>1e-4])
        print(thres)
        ops['Vcorr'] = V
    
    V = np.minimum(V, ops['Vcorr'])

    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(1, 1, 1)

    # find local maxima in a +/- d0 neighborhood

    i,j  = celldetect.localMax(V, rs<=d0, thres)
    if i.size==0:
        break

    # svd values of cell peaks
    new_codes = us[:,i,j]
    new_codes = new_codes / np.sum(new_codes**2, axis=0)**0.5

    for n in range(ncells, ncells+new_codes.shape[1]):
        ypix, xpix, goodi = celldetect.localRegion(i[n-ncells],j[n-ncells],dy,dx,Lyc,Lxc)
        # weights of svds surrounding max projected onto peak weights
        usub = Ucell[:, ypix, xpix]
        lam = np.maximum(0, new_codes[:,n-ncells].transpose() @ usub)
        lam[lam<lam.max()/5] = 0
        ipix = sub2ind((Lyc,Lxc), ypix, xpix)
        n0 = n*np.ones((goodi.sum(),),np.int32)
        
        mPix[goodi,n0] = ipix        
        mLam[goodi,n0] = lam
        # get connected region
        mLam[:,n] = celldetect.connectedRegion(mLam[:,n], rs, d0)
        mLam[:,n] = mLam[:,n] / np.sum(mLam[:,n]**2)**0.5
        # save lam in L, LtU, and LtS
        lam  = mLam[goodi,n]
        lam0 = np.zeros((1,Lyc*Lxc),np.float32)
        lam0[0,ipix] = lam
        if n>0:
            L = sparse.vstack([L, lam0])
        else:
            L[0,:] = lam0
        lam = np.expand_dims(lam,axis=0)
        LtU = np.append(LtU, lam @ np.resize(U[:,ypix,xpix], (nsvd,ypix.size)).transpose(), axis=0)
        LtS = np.append(LtS, lam @ S[:,ipix].transpose(), axis=0)

    ncells += new_codes.shape[1]

    # regression with neuropil
    L = sparse.csr_matrix(L)
    LtL = (L @ L.transpose()).toarray()
    cellcode = np.concatenate((LtL,LtS), axis=1)
    neucode  = np.concatenate((LtS.transpose(),StS), axis=1)
    codes = np.concatenate((cellcode, neucode), axis=0)
    Ucode = np.concatenate((LtU, StU),axis=0)
    codes = np.linalg.solve(codes + 1e-3*np.eye((codes.shape[0])), Ucode)
    neu   = codes[ncells:,:]
    codes = codes[:ncells,:]

    Ucell = U - np.resize(neu.transpose() @ S, U.shape) - np.resize(codes.transpose() @ L, U.shape)

    # reestimate masks
    L = sparse.lil_matrix((1,Lyc*Lxc),dtype=np.float32)    
    for n in range(0,ncells):
        goodi   = np.array((mPix[:,n]>=0).nonzero()).astype(np.int32)
        goodi   = goodi.flatten()
        n0      = n*np.ones(goodi.shape,np.int32)
        ipix    = mPix[goodi,n0].astype(np.int32)
        ypix,xpix = np.unravel_index(ipix.astype(np.int32), (Lyc,Lxc))
        usub    = (np.resize(Ucell[:,ypix,xpix],(nsvd,goodi.size)) 
                 + np.expand_dims(codes[n,:],axis=-1) @ np.expand_dims(mLam[goodi,n0],axis=0))

        lam = np.maximum(0, codes[n,:] @ usub)
        lam[lam<lam.max()/5] = 0
        mLam[goodi,n0] = lam
        mLam[:,n]  = celldetect.connectedRegion(mLam[:,n], rs, d0)
        mLam[:,n]  = mLam[:,n] / np.sum(mLam[:,n]**2)**0.5
        # save lam in L, LtU, and LtS
        lam = mLam[goodi,n]
        lam0 = np.zeros((1,Lyc*Lxc),np.float32)
        lam0[0,ipix] = lam
        if n>0:
            L = sparse.vstack([L, lam0])
        else:
            L[0,:] = lam0
        #L[n,:] = lam0
        lam = np.expand_dims(lam,axis=0)
        LtU[n,:]  = lam @ np.resize(U[:,ypix,xpix], (nsvd,ypix.size)).transpose()
        LtS[n,:]  = lam @ S[:,ipix].transpose()

        Ucell[:,ypix,xpix] = usub - (usub @ lam.transpose()) @ lam
    
    err = (Ucell**2).mean()

    print('%2.4f, %2.2f'%(err, toc(i0)))
    
    rgb = celldetect.drawClusters(r,mPix,mLam,Lyc,Lxc)    
    ax.imshow(rgb)        
    
    if it==0:
        Nfirst = i.size
    it += 1

In [ ]:
np.amax(V)

In [ ]:
plt.figure(figsize=(14,14))
#plt.imshow(sdmov.reshape((Lyc,Lxc)))
plt.imshow(V)
plt.show()
np.amax(V)

In [ ]:
mLam = mLam[:,:ncells]
mLam = mLam / np.expand_dims(mLam.sum(axis=1),axis=-1)
mPix = mPix[:,:ncells]   

Ucell = U - np.resize(neu.transpose() @ S, U.shape)
#ypix, xpix, goodi = celldetect.localRegion(i[n-ncells],j[n-ncells],dy,dx,Ly,Lx)

stat = celldetect.getStat(Lyc,Lxc,d0,mPix,mLam,codes,Ucell)    
stat = celldetect.getOverlaps(stat,Lyc,Lxc)
stat,cell_pix,cell_masks =celldetect.cellMasks(stat,Lyc,Lxc,False)
neuropil_masks = celldetect.neuropilMasks(ops,stat,cell_pix)
# add surround neuropil masks to stat
for n in range(ncells):
    stat[n]['ipix_neuropil'] = neuropil_masks[n,:,:].flatten().nonzero();

neuropil_masks = sparse.csc_matrix(np.resize(neuropil_masks,(-1,Lyc*Lxc)))
cell_masks     = sparse.csc_matrix(np.resize(cell_masks,(-1,Lyc*Lxc)))

In [ ]:
nimgbatch = 2000
nframes = int(ops['nframes'])
ny = ops['ny']
nx = ops['nx']
ncells = cell_masks.shape[0]
F    = np.zeros((ncells,nframes),np.float32)
Fneu = np.zeros((ncells,nframes),np.float32)

reg_file.close()
reg_file = open(ops['reg_file'], 'rb')
nimgbatch = int(nimgbatch)
block_size = ny*nx*nimgbatch*2
ix = 0
data = 1 
t0 = tic()
while data is not None:
    buff = reg_file.read(block_size)
    data = np.frombuffer(buff, dtype=np.int16, offset=0)
    nimgd = int(np.floor(data.size / (ny*nx)))
    if nimgd == 0:
        break
    data = np.reshape(data.astype(np.float32), (ny,nx,nimgd), order='F')
    data = np.transpose(data,(2,0,1))
    # crop data to valid region
    data = data[:, ops['yrange'][0]:ops['yrange'][-1]+1, ops['xrange'][0]:ops['xrange'][-1]+1]
    # resize data to be Ly*Lx by nimgd
    data = np.resize(data, (nimgd,Ly*Lx)).transpose()
    # compute cell activity
    inds = ix + np.arange(0,data.shape[1])
    F[:,inds]    = cell_masks @ data
    # compute neuropil activity
    Fneu[:,inds] = neuropil_masks @ data
    ix += data.shape[1]
    print(ix, toc(t0))
        
reg_file.close()


In [ ]:
plt.plot(F[503,:6000])

In [ ]:
def removeBadFrames(ops,F,Fneu):
    '''add nan's when bad frames occur (high movement frames) '''
    return F,Fneu

def activityStat(stat,F,Fneu):
    dF = F - 0.7 * Fneu
    sd = F.std(axis=1)
    sdN = Fneu.std(axis=1)
    std_cell = sd / sdN
    skew = np.skew(dF,axis=1)
    for n in range(ncells):
    return stat

In [ ]:
print(stat[2]['radius'])
inner_radius = 2
expanded_cell_pix = ndimage.grey_dilation(cell_pix, (inner_radius,inner_radius))
plt.subplot(121)
plt.imshow(cell_pix[:100,:100])
plt.subplot(122)
plt.imshow(expanded_cell_pix[:100,:100])
plt.show()
print(expanded_cell_pix[:100,:100])

In [ ]:
plt.imshow(cell_pix)

In [ ]:
n+=1
plt.imshow(neuropil_masks[n,:,:])
plt.plot(stat[n]['med'][1],stat[n]['med'][0],'o')
neuropil_masks[n,:,:].sum()

In [ ]:
n = 410
mask = np.zeros((Ly,Lx))
overlap = stat[n]['overlap']
ipix = stat[n]['ipix'][~overlap]
ypix = stat[n]['ypix'][~overlap]
xpix = stat[n]['xpix'][~overlap]
lam  = stat[n]['lam'][~overlap]
lam = lam / lam.sum()
print(lam)
mask[ypix,xpix] = lam
params = utils.fitMVGaus(ypix,xpix,lam,2)
plt.imshow(mask)
plt.plot(params[3][:,1],params[3][:,0])
#plt.plot(ypix[lam>.001],xpix[lam>.001],'.')
mu = params[0]
plt.xlim([mu[1]-10,mu[1]+10])
plt.ylim([mu[0]-10,mu[0]+10])
plt.show()